In [ ]:
start = "2025-05-01"
duration = 1

In [ ]:
from lsst.ts.xml.component_info import ComponentInfo
from lsst.summit.utils.efdUtils import makeEfdClient
from astropy.time import Time,TimeDelta
import os

In [ ]:
efd_client = makeEfdClient()
mtcamera_info = ComponentInfo("MTCamera", "sal")

start = Time(start, scale="utc")
end = start + TimeDelta(duration,"d")
with_data = 0
has_data = open('./existing_data.txt', 'w')
no_data = 0
missing_data = open('./missing_data.txt', 'w')
no_topic = 0
missing_topics = open('./missing_topic.txt', 'w')
data_dic = {}

for key in mtcamera_info.topics.keys():
    topic_info = mtcamera_info.topics[key]
    topic = topic_info.avro_subject.replace('-value', '')
    try:
        data = getEfdData(
            client=efd_client,
            topic=topic,
            columns=["*"],
            begin=start,
            end=end)
    except:
        no_topic += 1
        missing_topics.write(topic+'\n')
        continue
    if len(data) == 0:
        no_topic += 1
        missing_topics.write(topic+'\n')
        continue
    for name in topic_info.fields.keys():
        try:
            len_data = len(data[name])
        except:
            no_data += 1
            missing_data.write(f'{topic}.{name}\n')
            continue
        if len_data == 0:
            no_data += 1
        else:
            has_data.write(f'{topic}.{name}\n')
            with_data += 1
print(f"{no_topic} had no topic or topic contained no data.")
print(f"{with_data} queries returned data.")
print(f"{no_data} queries returned no data")
has_data.close()
missing_data.close()
missing_topics.close()
os.remove('./existing_data.txt')
os.remove('./missing_data.txt')
os.remove('./missing_topic.txt')